In [1]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import pandas_ta as ta
from sklearn.preprocessing import MinMaxScaler
#globals
TOTAL_FEATURES = 8
BACK_CANDLES = 25
FUTURE_DAYS = 1

def create_stock_dictionary():
    dictionary = {}
    current_directory = os.getcwd()
    folder_name = 'S&P500'
    folder_path = os.path.join(current_directory, folder_name)

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            name = file_name.split("_")[0]
            file_path = os.path.join(folder_path, file_name)
            dictionary[name] = pd.read_csv(file_path)
            
    return dictionary


In [2]:
stock_data = create_stock_dictionary()


In [11]:
stock_data = create_stock_dictionary()
n_days = BACK_CANDLES
m_days = FUTURE_DAYS

input_seqs = []
output_seqs = []
all_dates = pd.to_datetime([])  # Initialize an empty datetime index
for df in stock_data.values():
    all_dates = all_dates.union(pd.to_datetime(df['Datetime']))
all_dates = all_dates.sort_values()  # Sort the dates just in case

# Step 2: Reindex and fill missing data
for stock in stock_data:
    stock_data[stock] = stock_data[stock].set_index('Date')
    stock_data[stock].index = pd.to_datetime(stock_data[stock].index)  # Ensure index is datetime
    stock_data[stock] = stock_data[stock].reindex(all_dates, method='ffill')  # Use forward fill

scaler = MinMaxScaler()
#add features in here, also this basically just normalizes the entire dataframe without converting into a 2d array
for stock in stock_data:
    stock_data[stock].drop(['Datetime'], axis=1, inplace=True)
    stock_data[stock][stock_data[stock].columns] = scaler.fit_transform(stock_data[stock])

C:\Users\Rober\AppData\Local\Temp\ipykernel_2860\1043296246.py:9: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  all_dates = all_dates.union(pd.to_datetime(df['Datetime']))
C:\Users\Rober\AppData\Local\Temp\ipykernel_2860\1043296246.py:9: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  all_dates = all_dates.union(pd.to_datetime(df['Datetime']))
C:\Users\Rober\AppData\Local\Temp\ipykernel_2860\1043296246.py:9: FutureWarning: In a future version of 

KeyError: "None of ['Date'] are in the columns"

In [8]:
concatenated_data = np.hstack([stock_data[stock].iloc[:, 1:].values for stock in stock_data])
    # Create sequences
x_train = []
y_train = []
for i in range(len(concatenated_data) - n_days):
    x_train.append(concatenated_data[i:i + n_days])
    y_train.append(concatenated_data[i + n_days])

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 3348 and the array at index 12 has size 3341

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-01-03,14.621429,14.732143,14.607143,14.686786,12.433824,302220800
2012-01-04,14.642857,14.810000,14.617143,14.765714,12.500647,260022000
2012-01-05,14.819643,14.948214,14.738214,14.929643,12.639426,271269600
2012-01-06,14.991786,15.098214,14.972143,15.085714,12.771560,318292800
2012-01-09,15.196429,15.276786,15.048214,15.061786,12.751299,394024400
...,...,...,...,...,...,...
2022-12-23,130.919998,132.419998,129.639999,131.860001,130.959961,63814900
2022-12-27,131.380005,131.410004,128.720001,130.029999,129.142456,69007800
2022-12-28,129.669998,131.029999,125.870003,126.040001,125.179680,85438400
